Running the first section defines all the functions used in this workbook

In [60]:
import open3d as o3d
import numpy as np
import copy
import glob,os
import cloudComPy as cc
def prepare_dataset(voxel_size):
    print(":: Load two point clouds and disturb initial pose.")

    demo_icp_pcds = o3d.data.DemoICPPointClouds()
    source = o3d.io.read_point_cloud(demo_icp_pcds.paths[0])
    target = o3d.io.read_point_cloud(demo_icp_pcds.paths[1])
    trans_init = np.asarray([[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0],
                             [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
    source.transform(trans_init)
    draw_registration_result(source, target, np.identity(4))

    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source, target, source_down, target_down, source_fpfh, target_fpfh

def execute_fast_global_registration(source_down, target_down, source_fpfh,
                                     target_fpfh, voxel_size):
    distance_threshold = voxel_size * 0.5
    print(":: Apply fast global registration with distance threshold %.3f" \
            % distance_threshold)
    result = o3d.pipelines.registration.registration_fgr_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh,
        o3d.pipelines.registration.FastGlobalRegistrationOption(
            maximum_correspondence_distance=distance_threshold))
    return result
def draw_registration_result(source, target, zoom,up,front,lookat):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    # source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=zoom,
                                      up=up,
                                      front=front,
                                      lookat=lookat
                                      )
def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

def loadPointCloud_toO3D(fp):
    cctemp = cc.loadPointCloud(fp)
    xyz = cctemp.toNpArrayCopy()
    temp = o3d.geometry.PointCloud()
    temp.points = o3d.utility.Vector3dVector(xyz)
    return temp

This section then loads the clouds. note that it is written to look for files in "folder" with the "PCD" file type in  and don't start with "aligned..."

the last step is downsampling the base cloud

In [61]:
folder = r"X:\Projects\LidarScans\E\Individual_scans\E_20230727"
basefilepath = r"X:\PersonalShare\Luke\code\rockfall_processing\example\aligned_E_20210924.bin"
threshold = 0.1
icpv = 0.02

cloudpaths = os.listdir(folder)
clouds=[]
cloud_types = ('.e57','.pcd','.ply','.bin')
for cloudpath in cloudpaths:
    if cloudpath.endswith(cloud_types) and ('aligned' not in cloudpath):
        temp = loadPointCloud_toO3D(os.path.join(folder,cloudpath))
        temp = temp.voxel_down_sample(icpv)
        clouds.append(temp)
        print('loaded ',cloudpath)
base = loadPointCloud_toO3D(basefilepath)
base = base.voxel_down_sample(icpv)
print('loaded base and downsampled')

loaded  E_20230727_001.e57
loaded  E_20230727_003.e57
loaded  E_20230727_005.e57
loaded base and downsampled


In [ ]:
v = 0.85 #should be good for E
basedown,basefpfh = preprocess_point_cloud(base,v)
transforms = []
for i,cloud in enumerate(clouds):
    tempdown,tempfpfh = preprocess_point_cloud(cloud,v)
    result_fast = execute_fast_global_registration(tempdown, basedown,
                                            tempfpfh,basefpfh,
                                            v)
    tempdown.transform(result_fast.transformation) #tempdown is created just for quick visualization of results. it's transformed here.
    
    #you can write out the small roughly aligned cloud if you want here (but just as easy to quickly visualize using the code block after this one)
    # o3d.io.write_point_cloud(os.path.join(folder,'aligned_rough_'+str(i)+'.ply'),tempdown)


    cloud.transform(result_fast.transformation) #then we apply transformation to the full density cloud.
    transforms.append(result_fast.transformation)  




:: Downsample with a voxel size 0.850.
:: Estimate normal with search radius 1.700.
:: Compute FPFH feature with search radius 4.250.
:: Downsample with a voxel size 0.850.
:: Estimate normal with search radius 1.700.
:: Compute FPFH feature with search radius 4.250.
:: Apply fast global registration with distance threshold 0.425
:: Downsample with a voxel size 0.850.
:: Estimate normal with search radius 1.700.
:: Compute FPFH feature with search radius 4.250.
:: Apply fast global registration with distance threshold 0.425
:: Downsample with a voxel size 0.850.
:: Estimate normal with search radius 1.700.
:: Compute FPFH feature with search radius 4.250.
:: Apply fast global registration with distance threshold 0.425


In [ ]:
#if some or all of the rough alignments look good, you can run this to write them to files
for i,cloud  in enumerate(clouds):
    o3d.io.write_point_cloud(os.path.join(folder,'aligned_rough_full_'+str(i)+'.ply'),cloud)

you can use this to draw the registration result quickly 

In [66]:
for cloud in clouds:
    draw_registration_result(cloud,basedown,
                            zoom = 0.02,
                            up=[ -0.91284696632868534, 0.38836539258315206, -0.1260267348948248 ],
                            front = [ -0.053238637793158662, 0.19281089836405263, 0.97979059238082911 ],
                            lookat = [ -9.0112673659519142, 10.457115027765134, 1893.2071603034337 ])

**ICP**

with the roughly aligned clouds you can then run ICP for fine alignment

In [ ]:
#load roughly aligned clouds again
import numpy as np
import cloudComPy as cc

basefilepath = r"X:\PersonalShare\Luke\code\rockfall_processing\example\E_20210924_aligned.bin" #same as before
c1 = r"X:\PersonalShare\Luke\code\rockfall_processing\example\E_20240328_001_ex.bin"
c2 = r"X:\PersonalShare\Luke\code\rockfall_processing\example\E_20240328_003_ex.bin"
c3 = r"X:\PersonalShare\Luke\code\rockfall_processing\example\E_20240328_005_ex.bin"
fps = [c1,c2,c3]
ccClouds = []
for fp in fps:
    c = cc.loadPointCloud(fp)
    ccClouds.append(c)

basecloud = cc.loadPointCloud(basefilepath)

In [67]:
# OR RUN THIS IF YOU WANT TO KEEP ALL THE SAME POINT CLOUDS AS WERE USED BEFORE IN THE ROUGH ALIGNMENT
import cloudComPy as cc
ccClouds = []
for cloud in clouds:
    temp=cc.ccPointCloud()
    temp.coordsFromNPArray_copy(np.asarray(cloud.points,dtype=np.float32))
    ccClouds.append(temp)
    print('converted cloud from O3D to CloudComPy')

basecloud=cc.ccPointCloud()
basecloud.coordsFromNPArray_copy(np.asarray(base.points,dtype=np.float32))

converted cloud from O3D to CloudComPy
converted cloud from O3D to CloudComPy
converted cloud from O3D to CloudComPy


In [68]:
#%% ICP ALIGNMENT
numICP = 3
samplinglimits = [100000,200000,300000]
overlapRatios = [0.5,0.3,0.2]
for i,cloud in enumerate(ccClouds):
    for j in range(numICP):
        ICPres = cc.ICP(data=cloud,model=basecloud,minRMSDecrease = 1.e-5,
                    maxIterationCount = 20,
                    randomSamplingLimit = samplinglimits[j],
                    removeFarthestPoints = True,
                    method = cc.CONVERGENCE_TYPE.MAX_ERROR_CONVERGENCE,
                    finalOverlapRatio = overlapRatios[j], #MAY WANT TO ADJUST THIS BASED ON THE SITE CONDITIONS
                    adjustScale = False
                    )

        cloud.applyRigidTransformation(ICPres.transMat)
    res = cc.SavePointCloud(cloud,folder+f'\cloud_{i}_ICPalign.bin')
    print(res)

CC_FILE_ERROR.CC_FERR_NO_ERROR
CC_FILE_ERROR.CC_FERR_NO_ERROR
CC_FILE_ERROR.CC_FERR_NO_ERROR


In [69]:
# RUN THIS TO MERGE THE COMPONENT CLOUDS AND SAVE THE MERGED CLOUD
for i,cloud in enumerate(ccClouds):

    if i == 0:
        mergedCloud = cloud.cloneThis()
        
    else:
        mergedCloud.fuse(cloud)

assert mergedCloud.size() == np.sum([x.size() for x in ccClouds])

cc.SavePointCloud(mergedCloud,folder+'\mergedCloud.bin')

<CC_FILE_ERROR.CC_FERR_NO_ERROR: 0>

In [70]:
#RUN THIS TO DO A QUICK M3C2 COMPARISON BETWEEN THE NEWLY MERGED CLOUD AND THE BASE
if cc.isPluginM3C2():
    import cloudComPy.M3C2
M3C2params = r'X:\Projects\LidarScans\GW\Alignments\m3c2_params_GW_quickcompare.txt'
quickcompare = cc.M3C2.computeM3C2([basecloud,mergedCloud],M3C2params)
change = quickcompare.getScalarField(2).toNpArray()
filt = (change>-0.03) & (change < 0.03)
print(np.nanmean(change[filt]),np.nanstd(change[filt]))

cc.SavePointCloud(quickcompare,folder+'\quickcompare.bin')

0.0006735707 0.007013787


<CC_FILE_ERROR.CC_FERR_NO_ERROR: 0>

In the example, or test results for the quick compare were a mean of 0.0004 and a standard deviation of 0.00718, which is pretty good! Should still do a visual validation to make sure there aren't problem areas.